In [28]:
from yahooquery import Ticker

In [29]:
#market = Ticker(['fb', 'aapl', 'amzn', 'nflx'], asynchronous=True)

In [35]:
#import pandas_datareader as pdr
#nasdaqSymbols=pdr.get_nasdaq_symbols()['NASDAQ Symbol'].to_numpy().tolist()

In [36]:
import bs4 as bs
import pickle
import requests

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.rstrip()
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

In [37]:
market = Ticker(save_sp500_tickers(), asynchronous=True)

In [38]:
import pandas as pd

In [39]:
balanceSheets=market.balance_sheet(frequency="q")[[
    'asOfDate',
    'periodType',
    'InvestedCapital',
    'CashAndCashEquivalents',
    'CurrentDeferredLiabilities',
    'LongTermDebtAndCapitalLeaseObligation',
    'NonCurrentDeferredLiabilities',
    'OtherNonCurrentLiabilities',
    'CapitalStock',
    'CommonStock'
]]

In [87]:
balanceSheets.shape

(822, 10)

In [44]:
incomeStatements=market.income_statement(frequency="q")[[
    'asOfDate',
    'periodType',
    'EBIT'
]]

In [ ]:
valuationMeasures=market.valuation_measures[['asOfDate','periodType','MarketCap']]

In [21]:
#market.valuation_measures.__dict__

{'_is_copy': None,
 '_data': BlockManager
 Items: Index(['asOfDate', 'periodType', 'EnterpriseValue',
        'EnterprisesValueEBITDARatio', 'EnterprisesValueRevenueRatio',
        'ForwardPeRatio', 'MarketCap', 'PbRatio', 'PeRatio', 'PegRatio',
        'PsRatio'],
       dtype='object')
 Axis 1: Index(['AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'ADM', 'ADM',
        'ADM', 'ADM', 'ADM', 'ADM', 'ADM', 'AME', 'AME', 'AME', 'AME', 'AME',
        'AME', 'APTV', 'APTV', 'APTV', 'APTV', 'APTV', 'APTV', 'COO', 'COO',
        'COO', 'COO', 'COO', 'COO', 'ED', 'ED', 'ED', 'ED', 'ED', 'ED'],
       dtype='object', name='symbol')
 FloatBlock: slice(2, 11, 1), 9 x 38, dtype: float64
 DatetimeBlock: slice(0, 1, 1), 1 x 38, dtype: datetime64[ns]
 ObjectBlock: slice(1, 2, 1), 1 x 38, dtype: object,
 '_item_cache': {},
 '_attrs': {}}

In [46]:
valuationMeasures=valuationMeasures[valuationMeasures["periodType"]=="3M"]

In [47]:
valuationMeasures=valuationMeasures.groupby(['symbol'])['MarketCap'].max().to_frame()

In [48]:
#valuationMeasures=valuationMeasures[valuationMeasures['MarketCap']<1000000000000]

In [49]:
valuationMeasures

,MarketCap
symbol,
A,4.649661e+10
AAL,1.532885e+10
AAP,1.342413e+10
ABBV,1.990378e+11
ABC,2.424286e+10
...,...
STZ,4.595258e+10
T,2.161235e+11
TECH,1.754010e+10


In [50]:
# 500mm market cap limit

In [51]:
#valuationMeasures = valuationMeasures[valuationMeasures['MarketCap']>500000000]

In [52]:
mergedBsIs = pd.merge(balanceSheets, incomeStatements, on=['asOfDate','symbol','periodType'])

In [53]:
mergedBsIsSum = pd.merge(mergedBsIs, valuationMeasures, on=['symbol']).fillna(0)

In [54]:
mergedBsIsSum = mergedBsIsSum.groupby(['symbol'])['asOfDate',
                                  'InvestedCapital',
                                  'CashAndCashEquivalents',
                                  'CurrentDeferredLiabilities',
                                  'LongTermDebtAndCapitalLeaseObligation',
                                  'NonCurrentDeferredLiabilities',
                                  'OtherNonCurrentLiabilities',
                                  'CapitalStock',
                                  'CommonStock',
                                  'EBIT',
                                  'MarketCap'].max()

/Users/vincentchen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [55]:
roic = mergedBsIsSum.EBIT / mergedBsIsSum.InvestedCapital

In [56]:
cash = mergedBsIsSum.CashAndCashEquivalents

In [57]:
debt = mergedBsIsSum.CurrentDeferredLiabilities+mergedBsIsSum.LongTermDebtAndCapitalLeaseObligation+mergedBsIsSum.NonCurrentDeferredLiabilities+mergedBsIsSum.OtherNonCurrentLiabilities

In [58]:
preferredequity = mergedBsIsSum.CapitalStock-mergedBsIsSum.CommonStock

In [59]:
networth = mergedBsIsSum.InvestedCapital+cash-debt-preferredequity

In [60]:
faustmannRatio = mergedBsIsSum.MarketCap/networth

In [61]:
#criteria: ROIC > 20%

In [62]:
dfRoic = roic[roic>0.001].to_frame()
dfFaustmannRatio = faustmannRatio[faustmannRatio>0].to_frame()
dfRoicToFaustmann = (roic/faustmannRatio).to_frame()

In [63]:
dfRoic.columns=['roic']
dfFaustmannRatio.columns=['faustmannRatio']
dfRoicToFaustmann.columns=['roicToFaustmann']

In [64]:
dfFaustmannRatio['faustmannRank'] = dfFaustmannRatio.rank(ascending=True)
dfFaustmannRatio.sort_values(by=['faustmannRank'])

,faustmannRatio,faustmannRank
symbol,,
BK,0.218483,1.0
C,0.290452,2.0
CMA,0.342125,3.0
BAC,0.552111,4.0
AIG,0.599508,5.0
...,...,...
ADBE,22.946998,108.0
AON,23.587482,109.0
CLX,32.946767,110.0


In [65]:
dfRoic['roicRank']=dfRoic.rank(ascending=False)
dfRoic.sort_values(by=['roicRank'])

,roic,roicRank
symbol,,
BIO,0.327340,1.0
AZO,0.226399,2.0
BBWI,0.222826,3.0
CE,0.206171,4.0
BBY,0.174983,5.0
...,...,...
AEP,0.019430,117.0
ED,0.019334,118.0
COO,0.017358,119.0


In [66]:
result = pd.merge(
    pd.merge(
        pd.merge(mergedBsIsSum,dfRoic,on=['symbol']),
        dfFaustmannRatio,
        on=['symbol']
    ),
    dfRoicToFaustmann,
    on=['symbol']
)
#.sort_values(['roicToFaustmann'],ascending=False)

In [67]:
result['sumRanks']=result['faustmannRank']+result['roicRank']
result.sort_values(by=['sumRanks'],ascending=True)

,asOfDate,InvestedCapital,CashAndCashEquivalents,CurrentDeferredLiabilities,LongTermDebtAndCapitalLeaseObligation,NonCurrentDeferredLiabilities,OtherNonCurrentLiabilities,CapitalStock,CommonStock,EBIT,MarketCap,roic,roicRank,faustmannRatio,faustmannRank,roicToFaustmann,sumRanks
symbol,,,,,,,,,,,,,,,,,
BIO,2021-09-30,1.534476e+10,8.599020e+08,4.246800e+07,1.874630e+08,3.569332e+09,2.288700e+08,3.000000e+03,3.000000e+03,5.022952e+09,2.227957e+10,0.327340,1.0,1.829714,20.0,0.178902,21.0
ALL,2021-06-30,3.607200e+10,7.090000e+08,0.000000e+00,7.996000e+09,1.355000e+09,0.000000e+00,2.179000e+09,9.000000e+06,3.390000e+09,3.874068e+10,0.093979,18.0,1.533677,13.0,0.061277,31.0
CINF,2021-09-30,1.276500e+10,1.085000e+09,0.000000e+00,8.480000e+08,1.495000e+09,0.000000e+00,3.970000e+08,3.970000e+08,1.330000e+09,1.878748e+10,0.104191,15.0,1.632700,19.0,0.063815,34.0
BKR,2021-09-30,2.090600e+10,4.382000e+09,3.623000e+09,6.754000e+09,1.860000e+08,1.431000e+09,0.000000e+00,0.000000e+00,1.589000e+09,2.124307e+10,0.076007,29.0,1.597944,17.0,0.047565,46.0
CI,2021-06-30,8.401800e+10,1.018200e+10,0.000000e+00,3.160600e+10,9.142000e+09,0.000000e+00,4.000000e+06,4.000000e+06,5.715000e+09,8.323108e+10,0.068021,34.0,1.557118,15.0,0.043684,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRL,2021-06-30,5.019548e+09,4.654110e+08,2.763100e+08,2.966724e+09,2.808180e+08,7.084300e+07,5.050000e+05,5.050000e+05,2.055740e+08,1.864064e+10,0.040955,75.0,9.861394,97.0,0.004153,172.0
BLL,2021-06-30,1.138000e+10,1.366000e+09,0.000000e+00,7.783000e+09,6.710000e+08,4.940000e+08,1.195000e+09,1.195000e+09,3.730000e+08,3.052443e+10,0.032777,90.0,8.036974,88.0,0.004078,178.0
AWK,2021-06-30,1.768800e+10,5.600000e+08,0.000000e+00,1.042200e+10,2.529000e+09,1.542000e+09,2.000000e+06,2.000000e+06,4.330000e+08,2.797375e+10,0.024480,110.0,7.449734,82.0,0.003286,192.0


In [68]:
result.sort_values(by=['sumRanks'],ascending=True).to_csv('austrian_10q.csv', sep='\t')

In [59]:
result.sort_values(by=['sumRanks'],ascending=True).head(20)

,asOfDate,InvestedCapital,CashAndCashEquivalents,CurrentDeferredLiabilities,LongTermDebtAndCapitalLeaseObligation,NonCurrentDeferredLiabilities,OtherNonCurrentLiabilities,CapitalStock,CommonStock,EBIT,MarketCap,roic,roicRank,faustmannRatio,faustmannRank,roicToFaustmann,sumRanks
symbol,,,,,,,,,,,,,,,,,
BIO,2020-09-30,9.264022e+09,8.403250e+08,3.373500e+07,1.895970e+08,1.842659e+09,1.820850e+08,3.000000e+03,3000.0,1.690189e+09,1.533845e+10,0.182447,9.0,1.952383,112.0,0.093448,121.0
CINF,2020-09-30,1.071200e+10,9.140000e+08,0.000000e+00,8.460000e+08,1.079000e+09,0.000000e+00,3.970000e+08,397000000.0,1.159000e+09,1.712894e+10,0.108196,47.0,1.765688,99.0,0.061277,146.0
MET,2020-09-30,9.366000e+10,2.440500e+10,0.000000e+00,1.769100e+10,1.133400e+10,0.000000e+00,1.200000e+07,12000000.0,5.865000e+09,4.665478e+10,0.062620,133.0,0.523976,13.0,0.119510,146.0
FOX,2020-09-30,1.873800e+10,5.061000e+09,2.420000e+08,8.398000e+09,0.000000e+00,3.360000e+08,6.000000e+06,6000000.0,1.578000e+09,2.245519e+10,0.084214,80.5,1.514888,76.5,0.055591,157.0
FOXA,2020-09-30,1.873800e+10,5.061000e+09,2.420000e+08,8.398000e+09,0.000000e+00,3.360000e+08,6.000000e+06,6000000.0,1.578000e+09,2.245519e+10,0.084214,80.5,1.514888,76.5,0.055591,157.0
HUM,2020-09-30,2.376500e+10,7.985000e+09,0.000000e+00,6.059000e+09,0.000000e+00,0.000000e+00,3.300000e+07,33000000.0,2.662000e+09,5.477480e+10,0.112013,41.0,2.132062,126.0,0.052538,167.0
ALL,2020-09-30,3.192800e+10,5.470000e+08,0.000000e+00,6.635000e+09,1.154000e+09,0.000000e+00,2.257000e+09,9000000.0,2.313000e+09,3.587155e+10,0.072444,109.0,1.598696,86.0,0.045315,195.0
BWA,2020-09-30,7.670000e+09,2.121000e+09,0.000000e+00,2.787000e+09,1.740000e+08,6.980000e+08,3.000000e+06,3000000.0,4.870000e+08,8.953959e+09,0.063494,131.0,1.460202,73.0,0.043483,204.0
KIM,2020-09-30,1.090687e+10,4.517960e+08,0.000000e+00,5.800081e+09,0.000000e+00,5.171850e+08,4.345000e+06,4325000.0,7.812440e+08,8.942888e+09,0.071629,110.0,1.773897,100.0,0.040379,210.0


In [86]:
market.valuation_measures.loc[['HRB']]

KeyError: "None of [Index(['HRB'], dtype='object', name='symbol')] are in the [index]"